# Implementation of N-Gram

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 50
Batch_Size = 64
epochs = 2
n_classes = 2

Creating N-Gram Vectors set

In [2]:
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www.","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
url_dataframe

,url,type
0,law.justia.com/cases/federal/appellate-courts/...,1
1,cheaproomsvalencia.com/Paypal/Support/ID-NUMB3...,0
2,maguroban.s41.xrea.com/theme/pastmodern.html,0
3,tolstoybookz.info/perturbation-theory-for-line...,1
4,nortel-vpn-client.software.informer.com/,1
...,...,...
784286,bioguide.congress.gov/scripts/biodisplay.pl?in...,1
784287,coppeliaescoladeballet.com.br/index.php?option...,0
784288,mobilbrikett.hu/index.php?option=com_content&v...,0
784289,skovboefterskole.dk//wp-includes/index.html,1


In [40]:
df_another = pd.read_csv("../dataset/df_another_validation.csv")
df_another['url'] = df_another['url'].str.replace("https://","")
df_another['url'] = df_another['url'].str.replace("http://","")
df_another['url'] = df_another['url'].str.replace(" ","")
df_another['url'] = df_another['url'].str.replace("www.","")
df_another['url'] = df_another['url'].str.strip()
df_another

,url,type
0,eforu.com/birthdays/1950.html,1
1,toolsupdate.org/,0
2,angelfire.com/d20/dndchamps/,1
3,investorrelations.hk/foundation/gdocs/index.htm,0
4,netronic.com/gantt/gantt-charts.html,1
...,...,...
549341,jhandiecohut.com/076wc,0
549342,dailymotion.com/video/x30sfe_marvin-gaye-live-...,1
549343,aol.sportingnews.com/soccer/story/2011-10-07/s...,1
549344,fanpix.net/gallery/casey-stengel-pictures.htm,1


In [3]:
# def remove_path(link):
#     return re.sub(r"/.*", "", link)

In [4]:
# url_dataframe['url'] = url_dataframe['url'].apply(remove_path)
# url_dataframe

In [5]:
x = url_dataframe['url'].str.len()
x.describe(percentiles=[0.25,0.5,0.75,0.9,0.98,0.99,0.9999,0.99999,1])

count      784291.000000
mean           52.289089
std            41.232579
min             1.000000
25%            27.000000
50%            39.000000
75%            65.000000
90%            96.000000
98%           171.000000
99%           218.000000
99.99%        623.713000
99.999%      1371.000000
100%         2175.000000
max          2175.000000
Name: url, dtype: float64

In [6]:
# long_links = [link for link in url_dataframe['url'] if len(link) > 250]
# long_links

In [7]:
import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

2023-02-22 05:16:58.135963: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 05:16:58.259727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-22 05:16:58.259750: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-22 05:16:58.281864: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-22 05:16:58.848811: W tensorflow/stream_executor/platform/de

In [41]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))
df_another = tf.data.Dataset.from_tensor_slices((df_another['url'].values, df_another['type'].values))

In [42]:
train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)
df_another = df_another.batch(Batch_Size)

In [10]:
Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

In [11]:
# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

In [12]:
max_tokens = len(Vectorize_Layer.get_vocabulary())
max_tokens

54272

In [13]:
Vectorize_Layer.get_vocabulary()

['',
 '[UNK]',
 'c o m',
 'h t m',
 'i n d',
 'n d e',
 't m l',
 'e n t',
 'p h p',
 'd e x',
 'i o n',
 's c o',
 't i o',
 'e c o',
 'e x p',
 'n c o',
 'x p h',
 'a r t',
 'i n g',
 'o m p',
 'o n t',
 'i t e',
 'o r g',
 'o n c',
 'c o n',
 'l o g',
 'v i e',
 'a g e',
 'n t e',
 'n e t',
 'o m c',
 'e r s',
 'a t e',
 't o r',
 'a t i',
 'i e w',
 't e r',
 'o z i',
 'm o z',
 't i c',
 'l i n',
 'e s t',
 't h e',
 'a n d',
 't e m',
 'r t i',
 'e d i',
 't e n',
 'm c o',
 's t a',
 'z i m',
 't c o',
 'p t i',
 '2 0 1',
 'i n e',
 'e m i',
 'l c o',
 'o m a',
 'a l l',
 'o p t',
 'm i n',
 'g i n',
 'c a t',
 'm i d',
 'o n e',
 'o m i',
 'i s t',
 'f o r',
 'r e s',
 'p r o',
 'i k i',
 'c l e',
 'w i k',
 'o m e',
 's t o',
 'p o p',
 'e s s',
 'i c l',
 'a c o',
 'm a r',
 'h p o',
 'e s c',
 'p a g',
 'n e w',
 's t e',
 'o g i',
 'a m p',
 'd i a',
 'e n e',
 'o r t',
 't i n',
 'y c o',
 'a m e',
 'i d 1',
 'm e n',
 'w a r',
 'r e a',
 'i n s',
 'o m s',
 'e r i',
 'r c

In [14]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

In [43]:
train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)
df_another = df_another.map(vectorize_text)

In [16]:
Vectorize_Layer("hellobrother.com")

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 813,  212,  803, 2030, 3710,  588, 1339, 1041,   42,  231,  125,
        100,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])>

In [44]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
df_another = df_another.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 64)            3473408   
                                                                 
 lstm (LSTM)                 (None, 50, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,543,681
Trainable params: 3,543,681
Non-

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
history = model.fit(
    train_ds,
    epochs=epochs)

Epoch 1/2
9804/9804 [==============================] - 594s 60ms/step - loss: 0.2197 - accuracy: 0.9038
Epoch 2/2
9804/9804 [==============================] - 578s 59ms/step - loss: 0.1476 - accuracy: 0.9384


In [21]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
evaluate_model(model, test_ds)


2451/2451 [==============================] - 37s 15ms/step
[1 1 1 ... 1 1 0]
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1

Recall: 0.9300837057484748
F1 Score: 0.9299847669173207


[array([[64153,  6461],
        [ 4506, 81739]]),
 0.9300837057484748,
 0.9301787428546774,
 0.9300837057484748,
 0.9299847669173207]

In [34]:
temp = pd.DataFrame({'url':["https://drive.google.com","https://mail.google.com","https://paypal.me", "https://google.com", "https://123.534.643.me","https://live.onedrive.com"],'type':[0,0,0,0,0,0]})

In [35]:
temp['url'] = temp['url'].str.replace("https://","")
temp['url'] = temp['url'].str.replace("http://","")
temp['url'] = temp['url'].str.replace(" ","")
temp['url'] = temp['url'].str.replace("www.","")
temp['url'] = temp['url'].str.strip()
temp = tf.data.Dataset.from_tensor_slices((temp['url'].values, temp['type'].values))
temp.batch(Batch_Size)
temp = temp.map(vectorize_text)
temp = temp.cache().prefetch(buffer_size=AUTOTUNE)
temp.element_spec

(TensorSpec(shape=(1, 50), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [37]:
y_pred = model.predict(temp)
y_pred = y_pred.flatten()
y_pred = [round(i) for i in y_pred]
y_pred

6/6 [==============================] - 0s 5ms/step


[0, 1, 0, 1, 0, 0]

In [45]:
evaluate_model(model, df_another)

8584/8584 [==============================] - 124s 14ms/step
[1 0 1 ... 1 1 1]
[1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 

Accuracy: 0.7586839623843625
Precision: 0.745694518156346
Recall: 0.7586839623843625
F1 Score: 0.7487954132082649


[array([[ 72878,  83544],
        [ 49022, 343902]]),
 0.7586839623843625,
 0.745694518156346,
 0.7586839623843625,
 0.7487954132082649]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from nltk.util import ngrams
import itertools
N_for_NGram = 3
Sequence_length = 100
Batch_Size = 64
epochs = 5
n_classes = 2
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
url_dataframe['url'] = url_dataframe['url'].str.strip()
print(url_dataframe)

import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

max_tokens = len(Vectorize_Layer.get_vocabulary())
print("max_tokens" = max_tokens)

print(Vectorize_Layer.get_vocabulary())

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

print(model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_ds,
    epochs=epochs)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    print(y_true)
    print(y_pred)
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
result = evaluate_model(model, test_ds)

import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)
    
with open("result.pkl", "wb") as f:
    pickle.dump(result, f)